## Import module

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## Import data

In [4]:
df = pd.read_excel('Ppc_campaign_performance_data.xlsx')

In [12]:
df.head()

,Campaign_ID,Budget,Clicks,CTR,CPC,Conversions,CPA,Conversion_Rate,Duration,Platform,Content_Type,Target_Age,Target_Gender,Region,Revenue,Spend,ROAS,Date,Impressions,ROI
0,C3578,6390,401,0.0461,15.94,174,36.72,0.4339,20,Instagram,Carousel,35-44,Male,Europe,27840,6453.9,4.31,2025-01-19,8698,3.31
1,C6702,9870,1286,0.2860,7.67,821,12.02,0.6384,28,LinkedIn,Text,55+,Male,Africa,128076,10067.4,12.72,2025-01-22,4496,11.72
2,C9725,7700,1684,0.2122,4.57,1060,7.26,0.6295,15,Instagram,Video,35-44,Other,North America,193980,7623.0,25.45,2024-07-23,7935,24.45
3,C9472,8420,444,0.0961,18.96,308,27.34,0.6937,25,Google,Text,25-34,Male,North America,24024,8504.2,2.82,2024-04-20,4620,1.82
4,C7601,8470,1912,0.3652,4.43,1428,5.93,0.7469,9,Google,Text,25-34,Other,Europe,277032,8046.5,34.43,2024-08-07,5235,33.43


## Cleaning data

### Kiểm tra kiểu dữ liệu

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Campaign_ID      1000 non-null   object        
 1   Budget           1000 non-null   int64         
 2   Clicks           1000 non-null   int64         
 3   CTR              1000 non-null   float64       
 4   CPC              1000 non-null   float64       
 5   Conversions      1000 non-null   int64         
 6   CPA              1000 non-null   float64       
 7   Conversion_Rate  1000 non-null   float64       
 8   Duration         1000 non-null   int64         
 9   Platform         1000 non-null   object        
 10  Content_Type     1000 non-null   object        
 11  Target_Age       1000 non-null   object        
 12  Target_Gender    1000 non-null   object        
 13  Region           1000 non-null   object        
 14  Revenue          1000 non-null   int64   

-> Không có giá trị null, kiểu dữ liệu tương đối chuẩn, có vẻ như dữ liệu đã được người cung cấp trước đó làm sạch.  
-> Ta sẽ đổi kiểu dữ liệu của Date sang datetime

In [17]:
df["Date"] = pd.to_datetime(df["Date"])

### Kiểm tra trùng lặp

In [ ]:

df["Campaign_ID"].duplicated().sum()

np.int64(46)

Nhưng có thực sự là vậy không ? Ta cần chú ý đến liệu có phải rằng chiến dịch sẽ được thực hiện dở dang chưa hoàn thành nốt không ?

In [14]:
df["Campaign_ID"].value_counts()

Campaign_ID
C5910    3
C5327    2
C2099    2
C6702    2
C9725    2
        ..
C4071    1
C2402    1
C3257    1
C4315    1
C9612    1
Name: count, Length: 954, dtype: int64

In [15]:
df[df["Campaign_ID"] == "C5910"] 

,Campaign_ID,Budget,Clicks,CTR,CPC,Conversions,CPA,Conversion_Rate,Duration,Platform,Content_Type,Target_Age,Target_Gender,Region,Revenue,Spend,ROAS,Date,Impressions,ROI
272,C5910,4310,577,0.1408,7.47,188,22.93,0.3258,7,Facebook,Carousel,18-24,Other,Africa,20868,4223.8,4.94,2024-08-15,4098,3.94
351,C5910,7760,1931,0.7453,4.02,1221,6.36,0.6323,29,YouTube,Image,18-24,Other,Europe,238095,8148.0,29.22,2024-06-29,2590,28.22
827,C5910,7160,293,0.2169,24.44,291,24.60,0.9932,17,LinkedIn,Video,55+,Other,Asia,49179,7374.8,6.67,2024-04-23,1350,5.67


-> Có vẻ cùng Campaign_ID thế nhưng

## Create new featuress

In [ ]:
# Thêm một vài cột
df["Profit"] = df["Revenue"] - df["Cost"]
df["ROI"] = df["ROAS"] - 1
